In [0]:
# In this notebook we preprocess the product_descriptions.csv and finally we save the preprocessed file as df_descr_kwd.pkl
# Finally we save a final preprocessed file that will be used in the feature engineering : df_descr_kwd.pkl

In [0]:
import pandas as pd

# print non truncated column info in pandas dataframe
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)

In [0]:
from nltk.tokenize import RegexpTokenizer

from nltk.corpus import stopwords

from nltk.stem.porter import *

## - Preprocess product_descriptions.csv

In [0]:
# read the the file with the product descriptions
df_descriptions = pd.read_csv(r'https://bitbucket.org/dimitrisor/nlp_coursework_relevance_score_prediction/raw/4b81408f87a4f72746c45d47c5c385437df1ec5b/product_descriptions.csv', encoding='latin-1')
# read the trainset
df_train = pd.read_csv(r'https://bitbucket.org/dimitrisor/nlp_coursework_relevance_score_prediction/raw/4b81408f87a4f72746c45d47c5c385437df1ec5b/train.csv', encoding='latin-1')

In [0]:
# keep only the description of the products that appear in the trainset
df_descriptions = df_descriptions[df_descriptions.product_uid.isin(df_train.product_uid.unique())]

In [0]:
df_descriptions.head(1)

,product_uid,product_description
0,100001,"Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws"


In [0]:
# convert to lower case
df_descriptions['PD_lower'] = df_descriptions['product_description'].apply(lambda text: text.lower())

In [0]:
# remove punctuation and tokenize
# create a new column with the tokens
tokenizer = RegexpTokenizer(r'\w+')
df_descriptions['PD_tokens'] = df_descriptions['PD_lower'].apply(lambda text: tokenizer.tokenize(text))

In [0]:
# remove stopwords
stop_words = set(stopwords.words('english'))
df_descriptions['PD_tokens_sw'] = df_descriptions['PD_tokens'].apply(lambda tokens: [i for i in tokens if i not in stop_words])

In [0]:
# create a column wiht the tokens as text (without stopwords)
df_descriptions['PD_text'] = df_descriptions['PD_tokens_sw'].apply(lambda tokens: ' '.join(tokens))

In [0]:
# stemming
stemmer = PorterStemmer()
df_descriptions['PD_stem'] = df_descriptions['PD_tokens_sw'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

In [0]:
# create a list of terms of 'PD_tokens' that contain numbers
def hasNumbers(text):
    term_list = text.split(' ')
    numerics = []
    for term in term_list:
        if any(char.isdigit() for char in term)==1:
            numerics.append(term)
    return numerics

# create a list of terms of 'product_title' that contain numbers
df_descriptions['PD_numerics'] = df_descriptions['PD_lower'].apply(lambda x: hasNumbers(x))

In [0]:
# create a list of terms of 'PD_tokens' that don't contain numbers
def has_not_Numbers(terms):
    #term_list = text.split(' ')
    nonnumerics = []
    for term in terms:
        if any(char.isdigit() for char in term)==1:
            continue
        else:
            nonnumerics.append(term)
    return nonnumerics

# create a list of terms of 'PD_tokens' that don't contain numbers
df_descriptions['PD_Non_numerics'] = df_descriptions['PD_tokens_sw'].apply(lambda x: has_not_Numbers(x))

In [0]:
# create a column with the text of each product tile without any numeric terms
df_descriptions['PD_Clean_text'] = df_descriptions['PD_Non_numerics'].apply(lambda x: ' '.join(x))

In [0]:
df_descriptions.head(1)

,product_uid,product_description,PD_lower,PD_tokens,PD_tokens_sw,PD_text,PD_stem,PD_numerics,PD_Non_numerics,PD_Clean_text
0,100001,"Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws","not only do angles make joints stronger, they also provide more consistent, straight corners. simpson strong-tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. some can be bent (skewed) to match the project. for outdoor projects or those where moisture is present, use our zmax zinc-coated connectors, which provide extra resistance against corrosion (look for a ""z"" at the end of the model number).versatile connector for various 90 connections and home repair projectsstronger than angled nailing or screw fastening alonehelp ensure joints are consistently straight and strongdimensions: 3 in. x 3 in. x 1-1/2 in.made from 12-gauge steelgalvanized for extra corrosion resistanceinstall with 10d common nails or #9 x 1-1/2 in. strong-drive sd screws","[not, only, do, angles, make, joints, stronger, they, also, provide, more, consistent, straight, corners, simpson, strong, tie, offers, a, wide, variety, of, angles, in, various, sizes, and, thicknesses, to, handle, light, duty, jobs, or, projects, where, a, structural, connection, is, needed, some, can, be, bent, skewed, to, match, the, project, for, outdoor, projects, or, those, where, moisture, is, present, use, our, zmax, zinc, coated, connectors, which, provide, extra, resistance, against, corrosion, look, for, a, z, at, the, end, of, the, model, number, versatile, connector, for, various, 90, connections, and, home, repair, projectsstronger, than, angled, nailing, or, screw, fastening, alonehelp, ensure, ...]","[angles, make, joints, stronger, also, provide, consistent, straight, corners, simpson, strong, tie, offers, wide, variety, angles, various, sizes, thicknesses, handle, light, duty, jobs, projects, structural, connection, needed, bent, skewed, match, project, outdoor, projects, moisture, present, use, zmax, zinc, coated, connectors, provide, extra, resistance, corrosion, look, z, end, model, number, versatile, connector, various, 90, connections, home, repair, projectsstronger, angled, nailing, screw, fastening, alonehelp, ensure, joints, consistently, straight, strongdimensions, 3, x, 3, x, 1, 1, 2, made, 12, gauge, steelgalvanized, extra, corrosion, resistanceinstall, 10d, common, nails, 9, x, 1, 1, 2, strong, drive, sd, screws]",angles make joints stronger also provide consistent straight corners simpson strong tie offers wide variety angles various sizes thicknesses handle light duty jobs projects structural connection needed bent skewed match project outdoor projects moisture present use zmax zinc coated connectors provide extra resistance corrosion look z end model number versatile connector various 90 connections home repair projectsstronger angled nailing screw fastening alonehelp ensure joints consistently straight strongdimensions 3 x 3 x 1 1 2 made 12 gauge steelgalvanized extra corrosion resistanceinstall 10d common nails 9 x 1 1 2 strong drive sd screws,"[angl, make, joint, stronger, also, provid, co

In [0]:
# save the file temporarily as pickle file
df_descriptions.to_pickle(r'C:\Users\Dimos\Desktop\MSc\Semester 4\NLP\Coursework\data\preprocessed\df_descriptions_prep.pkl')

## -- TFIDF of Descriptions --> Keywords

In [0]:
# we will extract 10 keywords of each product description by using tf-idf

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [0]:
corpus = df_descriptions.PD_Clean_text.values.tolist()

In [0]:
len(corpus)

54667

In [0]:
# vectorizer = TfidfVectorizer(min_df=1)
# X = vectorizer.fit_transform(corpus)
# idf = list(vectorizer.idf_)
# terms = vectorizer.get_feature_names()

In [0]:
# create the tfidf matrix
vectorizer = TfidfVectorizer(min_df=1)
X = vectorizer.fit_transform(corpus)
idf = list(vectorizer.idf_)
terms = vectorizer.get_feature_names()

# create a dataframe with the terms and the tfidf score
df_tfidf = pd.DataFrame({'Term':terms, 'Score':idf})

In [0]:
# sort values
df_tfidf = df_tfidf.sort_values(by=['Score'], ascending=False)
df_tfidf.head()

,Score,Term
65746,11.215887,interiorsonline
76455,11.215887,materialsminimizes
76465,11.215887,materialsnew
76464,11.215887,materialsnear
76463,11.215887,materialsmultiple


In [0]:
df_tfidf.shape

(139997, 2)

### --- Keywords in Product Descriptions based on TF-IDF

In [0]:
def find_keywords(tokens, df, n):
    try:
        df_keywords = df[df.Term.isin(tokens)]
        df_keywords = df_keywords.head(n)
        keywords = df_keywords.Term.values.tolist()
    except:
        keywords = []
    
    return keywords

In [0]:
# find the keywords of each description by using the df_tfidf dataframe
df_descriptions['Keywords_Descr'] = df_descriptions.apply(lambda x: find_keywords(x['PD_Non_numerics'], df_tfidf, 10),
                                                               axis=1)

In [0]:
df_descriptions.head(15)

,product_uid,product_description,PD_lower,PD_tokens,PD_tokens_sw,PD_text,PD_stem,PD_numerics,PD_Non_numerics,PD_Clean_text,Keywords_Descr
0,100001,"Not only do angles make joints stronger, they also provide more consistent, straight corners. Simpson Strong-Tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. Some can be bent (skewed) to match the project. For outdoor projects or those where moisture is present, use our ZMAX zinc-coated connectors, which provide extra resistance against corrosion (look for a ""Z"" at the end of the model number).Versatile connector for various 90 connections and home repair projectsStronger than angled nailing or screw fastening aloneHelp ensure joints are consistently straight and strongDimensions: 3 in. x 3 in. x 1-1/2 in.Made from 12-Gauge steelGalvanized for extra corrosion resistanceInstall with 10d common nails or #9 x 1-1/2 in. Strong-Drive SD screws","not only do angles make joints stronger, they also provide more consistent, straight corners. simpson strong-tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. some can be bent (skewed) to match the project. for outdoor projects or those where moisture is present, use our zmax zinc-coated connectors, which provide extra resistance against corrosion (look for a ""z"" at the end of the model number).versatile connector for various 90 connections and home repair projectsstronger than angled nailing or screw fastening alonehelp ensure joints are consistently straight and strongdimensions: 3 in. x 3 in. x 1-1/2 in.made from 12-gauge steelgalvanized for extra corrosion resistanceinstall with 10d common nails or #9 x 1-1/2 in. strong-drive sd screws","[not, only, do, angles, make, joints, stronger, they, also, provide, more, consistent, straight, corners, simpson, strong, tie, offers, a, wide, variety, of, angles, in, various, sizes, and, thicknesses, to, handle, light, duty, jobs, or, projects, where, a, structural, connection, is, needed, some, can, be, bent, skewed, to, match, the, project, for, outdoor, projects, or, those, where, moisture, is, present, use, our, zmax, zinc, coated, connectors, which, provide, extra, resistance, against, corrosion, look, for, a, z, at, the, end, of, the, model, number, versatile, connector, for, various, 90, connections, and, home, repair, projectsstronger, than, angled, nailing, or, screw, fastening, alonehelp, ensure, ...]","[angles, make, joints, stronger, also, provide, consistent, straight, corners, simpson, strong, tie, offers, wide, variety, angles, various, sizes, thicknesses, handle, light, duty, jobs, projects, structural, connection, needed, bent, skewed, match, project, outdoor, projects, moisture, present, use, zmax, zinc, coated, connectors, provide, extra, resistance, corrosion, look, z, end, model, number, versatile, connector, various, 90, connections, home, repair, projectsstronger, angled, nailing, screw, fastening, alonehelp, ensure, joints, consistently, straight, strongdimensions, 3, x, 3, x, 1, 1, 2, made, 12, gauge, steelgalvanized, extra, corrosion, resistanceinstall, 10d, common, nails, 9, x, 1, 1, 2, strong, drive, sd, screws]",angles make joints stronger also provide consistent straight corners simpson strong tie offers wide variety angles various sizes thicknesses handle light duty jobs projects structural connection needed bent skewed match project outdoor projects moisture present use zmax zinc coated connectors provide extra resistance corrosion look z end model number versatile connector various 90 connections home repair projectsstronger angled nailing screw fastening alonehelp ensure joints consistently straight strongdimensions 3 x 3 x 1 1 2 made 12 gauge steelgalvanized extra corrosion resistanceinstall 10d common nails 9 x 1 1 2 strong drive sd screws,"[angl, make, joint, stronger, a

In [0]:
# keep only 2 columns
df_descr_keywords = df_descriptions[['product_uid', 'Keywords_Descr']]
df_descr_keywords.head()

,product_uid,Keywords_Descr
0,100001,"[strongdimensions, alonehelp, projectsstronger, zmax, skewed, resistanceinstall, sd, bent, steelgalvanized, thicknesses]"
1,100002,"[representationscolors, storesonline, galloncreates, informationrevives, coatingresists, deckover, rejuvenating, splinters, finishcovers, tired]"
4,100005,"[handlemaintains, systemdue, qualifications, vero, multichoice, minimalistic, separatelyincludes, elsewhere, scalding, unbx]"
5,100006,"[microwaveincluded, needsaccupop, hassletimesavor, suremist, browningindustry, accupop, favoriteskids, clearsingle, cleanupcook, onceclick]"
6,100007,"[ledsdual, neededengineering, proofintegrated, ceilingul, designsealed, requiredtypical, operationblack, quantum, cad, ni]"


In [0]:
# save the file temporarily
df_descr_keywords.to_pickle(r'C:\Users\Dimos\Desktop\MSc\Semester 4\NLP\Coursework\data\preprocessed\df_descr_keywords.pkl')

## Merge descriptions and keywords

In [0]:
# read the file with the preprocessed descriptions
df_descriptions= pd.read_pickle(r'C:\Users\Dimos\Desktop\MSc\Semester 4\NLP\Coursework\data\preprocessed\df_descriptions_prep.pkl')

# read the file with the keywords of the descriptions
df_descr_keywords = pd.read_pickle(r'C:\Users\Dimos\Desktop\MSc\Semester 4\NLP\Coursework\data\preprocessed\df_descr_keywords.pkl')

In [0]:
df_descriptions2 = df_descriptions[['product_uid', 'PD_lower', 'PD_stem', 'PD_numerics']]
df_descriptions3 = df_descriptions2.merge(df_descr_keywords, left_on='product_uid', right_on='product_uid', how='left')
df_descriptions3.head(2)

In [0]:
# save the file that will be used in the feature engineering
df_descriptions3.to_pickle(r'C:\Users\Dimos\Desktop\MSc\Semester 4\NLP\Coursework\data\preprocessed\df_descr_kwd.pkl')